# [LG 에너지 솔루션]
## DX Expert 양성과정 WEEK 2
## Convolutional Neural Network 2 - (2)

## 강의 복습
Convolutional Neural Network - Part 2

* ADENDA 04 CNN for Time-Series Data
## 실습 요약
1. 본 실습에서는 1D CNN 모델을 활용하여 시계열 분류 문제를 풀이합니다.
2. 학습된 모델을 활용하여 평가를 진행합니다.


---

### STEP 0. 환경 구축하기
* 필요한 Library 들을 import 합니다

In [1]:
import os
import time
import copy
import pickle
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import sys
from matplotlib import pyplot as plt

#Check torch version & device
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.6.0].
device:[cuda:0].


In [2]:
# set random seed 

def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
    
set_seed(42)

---

### STEP 1. 데이터 준비하기
금일 실습에서는 Human Activity Recognition Data 데이터를 활용합니다.

데이터 설명

* Human Activity Recognition (HAR) Data는 30명의 실험자들이 각자 스마트폰을 허리에 착용하고 6가지 활동 (Walking, Walking Upstairs, Walking Downstairs, Sitting, Standing, Laying)을 수행할 때 측정된 센서 데이터로 구성된 데이터셋입니다. 해당 데이터셋은 총 561개의 변수로 이루어져 있으며, 전체 데이터 중 70%는 train 데이터이고 나머지 30%는 test 데이터로 구성됩니다. HAR Data를 활용한 시계열 분류 task는 다변량 시계열 데이터를 input으로 받아 이를 다음 6가지 활동 중 하나의 class로 분류하는 것을 목표로 합니다: 0(Walking), 1(Walking Upstairs), 2(Walking Downstairs), 3(Sitting), 4(Standing), 5(Laying).

변수 설명

* 독립변수(X): 여러 실험자에 대하여 561개의 변수를 281 시점동안 수집한 시계열 데이터 -> shape: (#실험자, 561, 281)
* 종속변수(Y): 시계열 데이터의 label - 0(Walking) / 1(Walking Upstairs) / 2(Walking Downstairs) / 3(Sitting) / 4(Standing) / 5(Laying)

데이터 출처

https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones#

In [ ]:
# github에서 데이터 불러오기
!git clone https://github.com/yukyunglee/LG_ES_CNN_2

In [ ]:
! unzip ./LG_ES_CNN_2/data/har-data.zip -d ./LG_ES_CNN_2/data

In [3]:
DATA_DIR = './LG_ES_CNN_2/data/har-data'
BATCH = 32
N_CLASS = 6
EPOCHS = 100
WINDOW_SIZE = 50

In [4]:
x = pickle.load(open(os.path.join(DATA_DIR, 'x_train.pkl'), 'rb'))
y = pickle.load(open(os.path.join(DATA_DIR, 'state_train.pkl'), 'rb'))

In [5]:
x.shape

(21, 561, 281)

In [6]:
y.shape

(21, 281)

In [7]:
def data_preprocess(data_dir):
    # data_dir에 있는 train/test 데이터 불러오기
    x = pickle.load(open(os.path.join(data_dir, 'x_train.pkl'), 'rb'))
    y = pickle.load(open(os.path.join(data_dir, 'state_train.pkl'), 'rb'))
    x_test = pickle.load(open(os.path.join(data_dir, 'x_test.pkl'), 'rb'))
    y_test = pickle.load(open(os.path.join(data_dir, 'state_test.pkl'), 'rb'))

    # train data를 시간순으로 8:2의 비율로 train/validation set으로 분할
    # train, validation, test data의 개수 설정
    n_train = int(0.8 * len(x))
    n_valid = len(x) - n_train
    n_test = len(x_test)
    # train/validation set의 개수에 맞게 데이터 분할
    x_train, y_train = x[:n_train], y[:n_train]
    x_valid, y_valid = x[n_train:], y[n_train:]

    return x_train, y_train, n_train, x_valid, y_valid, n_valid, x_test, y_test, n_test

### STEP 3. Pytorch Dataset 정의하기
* 딥러닝 프레임워크를 활용하여 학습하기위해서는 물리적인 데이터셋을 학습 가능한 형태로 바꾸어주어야 합니다
* 01번 실습자료와 달리 시계열 데이터는 데이터의 형태에 맞게 가공해주는 과정이 중요합니다
* 다변량 시계열 데이터를 어떻게 다루어 학습 가능한 형태로 만들어주는지에 집중하면 좋습니다

In [8]:
def create_dataset(data_output,window_size):
    x_train, y_train, n_train, x_valid, y_valid, n_valid, x_test, y_test, n_test = data_output

    # train/validation/test 데이터를 window_size 시점 길이로 분할
    datasets = []
    for set in [(x_train, y_train, n_train), (x_valid, y_valid, n_valid), (x_test, y_test, n_test)]:
        # 전체 시간 길이 설정
        T = set[0].shape[-1]
        print(set[0].shape)
        # 전체 X 데이터를 window_size 크기의 time window로 분할
        windows = np.split(set[0][:, :, :window_size * (T // window_size)], (T // window_size), -1)
        windows = np.concatenate(windows, 0)
        print(windows.shape)
        # 전체 y 데이터를 window_size 크기에 맞게 분할
        labels = np.split(set[1][:, :window_size * (T // window_size)], (T // window_size), -1)
        labels = np.round(np.mean(np.concatenate(labels, 0), -1))
        # 분할된 time window 단위의 X, y 데이터를 tensor 형태로 축적
        datasets.append(torch.utils.data.TensorDataset(torch.Tensor(windows), torch.Tensor(labels)))
        print("="*10)
    return datasets


In [9]:
data_output = data_preprocess(data_dir=DATA_DIR)
datasets = create_dataset(data_output, window_size=WINDOW_SIZE)

# train/validation/test DataLoader 구축
trainset, validset, testset = datasets[0], datasets[1], datasets[2]
train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=BATCH, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=BATCH, shuffle=True)

(16, 561, 281)
(80, 561, 50)
(5, 561, 281)
(25, 561, 50)
(9, 561, 288)
(45, 561, 50)


#### STEP 4. 모델 정의 및 학습 준비하기
* 시계열 데이터를 위한 1D Convolution 모델을 정의하고 학습을 진행합니다
* 사전 학습 모델을 사용하지 않고 모델링을 정의하는 방법에 대해 살펴보겠습니다

In [10]:
# 1-dimensional convolution layer로 구성된 CNN 모델
# 2개의 1-dimensional convolution layer와 1개의 fully-connected layer로 구성되어 있음
class CNN_1D(nn.Module):
    def __init__(self, num_classes):
        super(CNN_1D, self).__init__()
        # 첫 번째 1-dimensional convolution layer 구축
        self.layer1 = nn.Sequential(
            nn.Conv1d(561, 64, kernel_size=3),
            nn.ReLU(),
            nn.AvgPool1d(2)
        )
        # 두 번째 1-dimensional convolution layer 구축
        self.layer2 = nn.Sequential(
            nn.Conv1d(64, 64, kernel_size=3),
            nn.ReLU(),
            nn.AvgPool1d(2)
        )
        # fully-connected layer 구축
        self.fc = nn.Linear(64 * 11, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [11]:
# 1D CNN 구축
model = CNN_1D(num_classes=N_CLASS)
model = model.to(device)
print(model)

CNN_1D(
  (layer1): Sequential(
    (0): Conv1d(561, 64, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  )
  (layer2): Sequential(
    (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  )
  (fc): Linear(in_features=704, out_features=6, bias=True)
)


In [12]:
# SGD optimizer 구축하기
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# loss function 설정
criterion = nn.CrossEntropyLoss()

In [13]:
def train_model(model, dataloaders, criterion, num_epochs, optimizer):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(EPOCHS):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # 각 epoch마다 순서대로 training과 validation을 진행
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 training mode로 설정
            else:
                model.eval()   # 모델을 validation mode로 설정

            running_loss = 0.0
            running_corrects = 0
            running_total = 0

            # training과 validation 단계에 맞는 dataloader에 대하여 학습/검증 진행
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device, dtype=torch.long)

                # parameter gradients를 0으로 설정
                optimizer.zero_grad()

                # forward
                # training 단계에서만 gradient 업데이트 수행
                with torch.set_grad_enabled(phase == 'train'):
                    # input을 model에 넣어 output을 도출한 후, loss를 계산함
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # output 중 최댓값의 위치에 해당하는 class로 예측을 수행
                    _, preds = torch.max(outputs, 1)

                    # backward (optimize): training 단계에서만 수행
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # batch별 loss를 축적함
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_total += labels.size(0)

            # epoch의 loss 및 accuracy 도출
            epoch_loss = running_loss / running_total
            epoch_acc = running_corrects.double() / running_total

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # validation 단계에서 validation loss가 감소할 때마다 best model 가중치를 업데이트함
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    # 전체 학습 시간 계산
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # validation loss가 가장 낮았을 때의 best model 가중치를 불러와 best model을 구축함
    model.load_state_dict(best_model_wts)
    
    # best model 가중치 저장
    # torch.save(best_model_wts, '../output/best_model.pt')
    return model, val_acc_history

In [14]:
# trining 단계에서 사용할 Dataloader dictionary 생성
dataloaders_dict = {
    'train': train_loader,
    'val': valid_loader
}

In [15]:
# 모델 학습
model, val_acc_history = train_model(model, dataloaders_dict, criterion, EPOCHS, optimizer)

Epoch 1/100
----------
train Loss: 1.7940 Acc: 0.0750
val Loss: 1.7858 Acc: 0.3200

Epoch 2/100
----------
train Loss: 1.7800 Acc: 0.3500
val Loss: 1.7635 Acc: 0.5600

Epoch 3/100
----------
train Loss: 1.7534 Acc: 0.4250
val Loss: 1.7263 Acc: 0.5200

Epoch 4/100
----------
train Loss: 1.7151 Acc: 0.4125
val Loss: 1.6654 Acc: 0.5200

Epoch 5/100
----------
train Loss: 1.6518 Acc: 0.4125
val Loss: 1.5782 Acc: 0.5200

Epoch 6/100
----------
train Loss: 1.5608 Acc: 0.4125
val Loss: 1.4583 Acc: 0.5200

Epoch 7/100
----------
train Loss: 1.4438 Acc: 0.4125
val Loss: 1.3427 Acc: 0.5200

Epoch 8/100
----------
train Loss: 1.3369 Acc: 0.4125
val Loss: 1.2940 Acc: 0.5200

Epoch 9/100
----------
train Loss: 1.2888 Acc: 0.4125
val Loss: 1.3152 Acc: 0.5200

Epoch 10/100
----------
train Loss: 1.2557 Acc: 0.4125
val Loss: 1.3479 Acc: 0.5200

Epoch 11/100
----------
train Loss: 1.2283 Acc: 0.4250
val Loss: 1.3776 Acc: 0.5600

Epoch 12/100
----------
train Loss: 1.1899 Acc: 0.5000
val Loss: 1.3230 Ac

In [16]:
def test_model(model, test_loader):
    model.eval()   # 모델을 validation mode로 설정
    
    # test_loader에 대하여 검증 진행 (gradient update 방지)
    with torch.no_grad():
        corrects = 0
        total = 0
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device, dtype=torch.long)

            # forward
            # input을 model에 넣어 output을 도출
            outputs = model(inputs)

            # output 중 최댓값의 위치에 해당하는 class로 예측을 수행
            _, preds = torch.max(outputs, 1)

            # batch별 정답 개수를 축적함
            corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

    # accuracy를 도출함
    test_acc = corrects.double() / total
    print('Testing Acc: {:.4f}'.format(test_acc))

In [17]:
# 모델 검증 (Acc: 0.8222)
test_model(model, test_loader)

Testing Acc: 0.8222
